In [ ]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
need to add in qc and mean

In [ ]:
def get_data_from_csv(filename):
    data = pd.read_csv(filename)
    d={}
    d['amps'] = []
    d['spike_times_relative_to_expt'] = []
    d['train_id'] = []
    d['expt_start_time'] = []
    d['post_id'] = data['post_id']
    d['pre_id'] = data['pre_id']
    d['uid'] = data['uid']
    
    #convert weird string from cvs to arrays
    for ii in range(len(data)):
        d['amps'].append(np.fromstring(data['deconvolved_amps'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=float, sep=' '))
        d['spike_times_relative_to_expt'].append(np.fromstring(data['spike_times_relative_to_experiment'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=float, sep=' '))
        d['expt_start_time'].append(d['spike_times_relative_to_expt'][-1][0])
        d['train_id'].append(np.fromstring(data['post_syn_rec_id'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=int, sep=' '))
        if (len(d['amps'][-1]) != len(d['spike_times_relative_to_expt'][-1])) or \
            (len(d['amps'][-1]) != len(d['train_id'][-1])):
            raise Exception('string to array conversion probably went wrong')
    return d

data=get_data_from_csv('pv_tlx_to_sst.csv')

In [ ]:
for ii in range(len(data)):
    plt.figure(figsize=(15,10))
    
    # plot whole experiment
    sample=range(0,len(data['amps']))
    plt.plot(data['spike_times_relative_to_expt'][ii][sample], data['amps'][ii][sample], '.-', ms=20)
    # for ii, (p,t,a) in enumerate(zip(pulse_ids[sample], spike_times_relative_to_expt[sample], amplitudes[sample]*1e3)):
    #     plt.annotate(str(ii), xy=(t,a), textcoords='data')
    plt.ylabel('Deconvolution Arbitrary Units')
    plt.xlabel('time (unknown units)')
    plt.title('Deconvolution Amplitude\n%.3f, pre %i, post %i' %(data['uid'][ii], data['pre_id'][ii], data['post_id'][ii]))
    
    # individual trains
    plt.figure(figsize=(15,10))
    start_train_index = 0
    end_train_index = 0
    train_id = data['train_id'][ii]
    for jj in range(1, len(train_id)):
        if (train_id[jj-1] != train_id[jj]):
            end_train_index = jj-1  
            #make the plot
            sample = range(start_train_index, end_train_index+1) #+1 due to range
            plt.plot(data['spike_times_relative_to_expt'][ii][sample]-data['spike_times_relative_to_expt'][ii][start_train_index], data['amps'][ii][sample], '.-', ms=20)
            start_train_index = jj
    
    plt.show()